<a href="https://colab.research.google.com/github/BrittonWinterrose/DS/blob/master/Flocks_TwitterAPI_Call.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Twitter API Search

In [2]:
pip install python-twitter

    100% |████████████████████████████████| 71kB 4.3MB/s 


In [0]:
# -*- coding: utf-8 -*-

import twitter
import json

# load Twitter API credentials

with open('twitter_credentials.json') as cred_data:
    info = json.load(cred_data)
    consumer_key = info['CONSUMER_KEY']
    consumer_secret = info['CONSUMER_SECRET']
    access_key = info['ACCESS_KEY']
    access_secret = info['ACCESS_SECRET']


    
def get_search(query):
	
	
  t = twitter.Api(consumer_key=consumer_key,
                consumer_secret=consumer_secret,
                access_token_key=access_key,
                access_token_secret=access_secret,
                tweet_mode='extended',
                sleep_on_rate_limit=False,)
  
  
   #initialize a list to hold all the tweepy Tweets
  all_responses=[]
  

  new_responses = t.GetSearch(term=query, count=100, lang='en',result_type='recent')
  
  all_responses.extend(new_responses)
  
  
  oldest = all_responses[-1].id - 1
  
  
  #THIS WILL NEED TO BE CHANGED
  while len(all_responses) < 100:

        print( "getting tweets before %s" % (oldest))
        
        #all subsiquent requests use the max_id param to prevent duplicates
        new_responses = t.GetSearch(term=query,count=100,max_id=oldest, lang='en',result_type='recent')
        
        #save most recent tweets
        all_responses.extend(new_responses)
        
        #update the id of the oldest tweet less one
        oldest = all_responses[-1].id - 1
        
        print( "...%s tweets downloaded so far" % (len(all_responses)))
	
  
  
  out = []
  
  for tweet in all_responses:
    if tweet.retweeted_status:
      out_list = [tweet.id_str, tweet.created_at, tweet.retweeted_status.full_text]
      out.append(out_list)
    else:
      out_list = [tweet.id_str, tweet.created_at, tweet.full_text]
      out.append(out_list)
  
  
  
  
  #out = [[tweet.id_str, tweet.created_at, tweet.full_text] for tweet in all_responses]
  
  
  #format tweet information    
  labels={0: 'tweet_ID',1: 'tweet_date', 2: 'tweet_text'}
  tweet_dict = [{labels[idx]:val for idx,val in enumerate(item)} for item in out]
  
  
  return tweet_dict


In [4]:
%time
search = get_search('Lambda School')

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 6.2 µs


In [5]:
search

[{'tweet_ID': '1118663428753608704',
  'tweet_date': 'Wed Apr 17 23:51:59 +0000 2019',
  'tweet_text': "@nnanna I've been following what Lambda School has been doing and this approach will change the type of opportunities available for a lot of people."},
 {'tweet_ID': '1118662075843272705',
  'tweet_date': 'Wed Apr 17 23:46:37 +0000 2019',
  'tweet_text': 'First Andela, now Lambda School.. this is how Nigeria’s economy gets turned around https://t.co/1fh5SYLdWf'},
 {'tweet_ID': '1118660454212145152',
  'tweet_date': 'Wed Apr 17 23:40:10 +0000 2019',
  'tweet_text': "Lambda School now live in:\n* United States\n* European Union (https://t.co/k7pJenVUht)\n* Africa (Ghana, Kenya, Nigeria, Cote D'Ivoire and South Africa) https://t.co/nEO4svBt2c\n* Test students now in India, Canada, Southeast Asia, Latin America\n\nSoon."},
 {'tweet_ID': '1118658364995723266',
  'tweet_date': 'Wed Apr 17 23:31:52 +0000 2019',
  'tweet_text': "It’s true, Lambda School is opening up (at small scale) to stud

In [0]:
import pandas as pd
import numpy as np

df = pd.DataFrame.from_dict(search)

In [19]:
#df = df.infer_objects()
df.tweet_ID= pd.to_numeric(df.tweet_ID)
df.tweet_date=pd.to_datetime(df.tweet_date)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 3 columns):
tweet_ID      100 non-null int64
tweet_date    100 non-null datetime64[ns]
tweet_text    100 non-null object
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 2.4+ KB


In [0]:
tweets = df[["tweet_ID", "tweet_date"]].to_json("tweets.json",orient="records")